In [3]:
import gym
import matplotlib.pyplot as plt
import collections
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from tensorflow import keras
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.python.client import device_lib
from random import sample

from hyperopt import fmin, tpe, hp, Trials, space_eval, STATUS_OK
import pickle

import pandas as pd
from openpyxl import load_workbook

import xgboost as xgb
import shap

import cv2

print(tf.__version__)
print(device_lib.list_local_devices())

c:\Users\Go\miniconda3\envs\tensorgpu\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
c:\Users\Go\miniconda3\envs\tensorgpu\lib\site-packages\keras_preprocessing\image\utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
c:\Users\Go\miniconda3\envs\tensorgpu\lib\site-packages\keras_preprocessing\image\utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
c:\Users\Go\miniconda3\envs\tensorgpu\lib\site-packages\keras_preprocessing\image\utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
c:\Users\Go\minic

2.8.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2424923833778238398
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3667263488
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1073487036815429889
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


### mobnet preprocessing 사용

In [128]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  
])
model.compile()

In [140]:
def imageresize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    return np.array(img)

def state_to_xgbinput(img, pre_model):
    resized_img = imageresize(img)
    input = pre_model(resized_img.reshape([-1,224,224,3])).numpy()
    return input

In [141]:
def make_model(states, actions, model_name):
    # x = np.concatenate(states, axis = 0).reshape([-1,4])
    x = np.array(states).reshape([-1,1280])
    y = np.array(actions).reshape([-1,1])
    dtrain = xgb.DMatrix(x, y)
    if model_name == None:
        model = xgb.train(params, dtrain)
    else:
        model = xgb.train(params, dtrain, xgb_model=model_name)
    return model

def get_action(xgbmodel, input_state,):
    input = xgb.DMatrix(input_state)
    out = xgbmodel.predict(input)

    if out > 0.5:
        a = 1
    else:
        a = 0
    return a

def get_action_egreedy(xgbmodel, input_state,eps):
    if np.random.choice([True, False],p = [eps,1-eps]):
        a = np.random.choice([0, 1],p = [0.5,0.5])
    else:
        a = get_action(xgbmodel,input_state)
    return a

In [142]:
params = {
    "tree_method": "hist", 
    "objective": 'binary:logistic',
    # "objective": 'binary:logistic',
    # "objective": 'multi:softprob',
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "single_precision_histogram" : True,
          }

env = gym.make('Pong-v0')
s = env.reset()
input = state_to_xgbinput(s, model)
xgb_clf = make_model(input,[[1]], model_name=None)
xgb_clf.save_model("model_pong.json")

epsilon = 1
eps_min = 0.1

total_r_list = []
states_list = []
actions_list = []



for _ in range(500):

    if epsilon > eps_min:
        epsilon = epsilon*0.99
    else:
        epsilon = eps_min
    
    states = []
    actions = []
    total_r = 0
    s = env.reset()
    input = state_to_xgbinput(s, model)

    a = get_action_egreedy(xgb_clf, input,epsilon)

    states.append(input)
    actions.append(a)
    while True:
        ns, r, done, _ = env.step(a)
        # total_r += r
        total_r += 1 #오래할수록 크게나오도록

        if r != 0:
            print(r)

        if done:
            print('ok')
            print(total_r)
            total_r_list.append(total_r)
            states_list.append(states)
            actions_list.append(actions)
            break
        else:
            s = ns
            input = state_to_xgbinput(s, model)
            a = get_action_egreedy(xgb_clf, input,epsilon)
            states.append(input)
            actions.append(a)

    avg = sum(total_r_list)/len(total_r_list)
    print(avg)
    train_x = []
    train_y = []

    for i in range(len(total_r_list)):
        if total_r_list[i] >= avg:
            train_x.extend(states_list[i])
            train_y.extend(actions_list[i])
    xgb_clf = make_model(train_x,train_y,"model_pong.json")

WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v5` with the environment ID `ALE/Pong-v5`.


-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1017
1017.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1022
1019.5
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1032
1023.6666666666666
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1032
1025.75
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1006
1021.8
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1017
1021.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1022
1021.1428571428571
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
ok
1020
1021.0
-1.0
-1.0
-1.0


KeyboardInterrupt: 

### 3d to 1d 전처리

In [16]:
def image_to_input(img):
    s2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    s3 = cv2.resize(s2, dsize=(80, 80), interpolation=cv2.INTER_AREA)
    s4 = s3.reshape(1,-1)
    return s4

def state_to_xgbinput(img, pre_model):
    resized_img = imageresize(img)
    input = pre_model(resized_img.reshape([-1,224,224,3])).numpy()
    return input

In [33]:
def make_model(states, actions, model_name):
    # x = np.concatenate(states, axis = 0).reshape([-1,4])
    x = np.array(states).reshape([-1,6400])
    y = np.array(actions).reshape([-1,1])
    dtrain = xgb.DMatrix(x, y)
    if model_name == None:
        model = xgb.train(params, dtrain)
    else:
        model = xgb.train(params, dtrain, xgb_model=model_name)
    return model

def get_action(xgbmodel, input_state,):
    input = image_to_input(input_state)
    input = xgb.DMatrix(input)
    out = xgbmodel.predict(input)

    if out > 0.5:
        a = 1
    else:
        a = 0
    return a

def get_action_egreedy(xgbmodel, input_state,eps):
    if np.random.choice([True, False],p = [eps,1-eps]):
        a = np.random.choice([0, 1],p = [0.5,0.5])
    else:
        a = get_action(xgbmodel,input_state)
    return a

In [92]:
params = {
    "tree_method": "hist", 
    "objective": 'binary:logistic',
    # "objective": 'binary:logistic',
    # "objective": 'multi:softprob',
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "single_precision_histogram" : True,
          }

env = gym.make('Pong-v0')
s = env.reset()
input = image_to_input(s)
xgb_clf = make_model(input,[[1]], model_name=None)
xgb_clf.save_model("model_pong2.json")

epsilon = 1
eps_min = 0.1


total_r_list = []
states_list = []
actions_list = []

for _ in range(40000):
    for _ in range(5):
        act_pass_frame = 4
        if epsilon > eps_min and avg > -21:
            epsilon = epsilon*0.995
        else:
            pass
        
        states = []
        actions = []
        total_r = 0
        s = env.reset()
        a = get_action_egreedy(xgb_clf, s,epsilon)

        states.append(image_to_input(s))
        actions.append(a)
        while True:
            ns, r, done, _ = env.step(a)
            # total_r += r
            total_r += r 

            

            if done:
                total_r_list.append(total_r)
                states_list.append(states)
                actions_list.append(actions)
                break
            else:
                s = ns
                if act_pass_frame == 0 :
                    a = get_action_egreedy(xgb_clf, s,epsilon)
                    act_pass_frame = 4
                act_pass_frame -= 1
                states.append(image_to_input(s))
                actions.append(a)

    avg = sum(total_r_list)/len(total_r_list)
    print(avg)
    print(epsilon)
    train_x = []
    train_y = []

    for i in range(len(total_r_list)):
        if total_r_list[i] > avg:
            train_x.extend(states_list[i])
            train_y.extend(actions_list[i])
    if len(train_y) != 0:
        xgb_clf = make_model(train_x,train_y,"model_pong2.json")

-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1
-21.0
1


KeyboardInterrupt: 